# Building Ships

Ships are large structures that can be built in the `industrial_complex`. They consume resources to make and must be stored in a `shipyard`. 

In [1]:
import sys, os
import pandas as pd
import altair as alt
sys.path.append('../..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 

nest_asyncio.apply()

PII_cols = ['username','userguid','owner']

In [24]:
from app.functions import configurations
from app.objects import baseobjects

In [25]:
from app.connectors.cmdb_graph import CosmosdbClient
c = CosmosdbClient()

industrial_complex `has_buttons`, which can `build_ship`. Ships are built from `designs`, which are also in the config. 

In [54]:
ship_configurations = configurations.get_ship_configurations()['ship']
ship_configurations.keys()

dict_keys(['explanation', 'components', 'designs'])

A ships effort is the sum of it's components

In [55]:
probe = ship_configurations['designs']['probe']
probe

{'type': 'probe',
 'label': 'ship',
 'name': 'Probe',
 'description': 'One way scanning machine to observe a location. Requires ',
 'components': ['engine', 'scanner']}

# The Ship object

In [186]:
class Component(baseobjects.Baseobject):
    def __init__(self, config):
        super().__init__()
        self.label = "component"
        self.config = config
        self.type = config['type']
        self.name = config['name']
    

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.config.items():
            fund[k] = v
        fund['type'] = self.config['type']
        return fund


In [195]:
class Design(baseobjects.Baseobject):
    def __init__(self, config):
        super().__init__()
        self.label = "design"
        self.config = config
        self.type = config['type']
        self.name = config['name']
        self.design_type = config['label']
    

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.config.items():
            fund[k] = v
        fund['type'] = self.config['type']
        fund['label'] = self.label
        return fund


In [ ]:
class Ship(baseobjects.Baseobject):
    def __init__(self, design, component_configurations):
        super().__init__()
        self.label = "ship"
        # TODO: Naming for ships
        self.name = design['name']
        self.type = design['type']
        self.design = Design(design)
        self.components = [Component(component_configurations[c]) for c in design['components']]
        self.stats = {}
        self.build_stats()
    
    def build_stats(self):
        for i in self.components:
            c = i.get_data()
            if 'augments_ship_stats' in c.keys():
                for k,v in c['augments_ship_stats'].items():
                    if k in self.stats.keys():
                        self.stats[k] += v
                    else:
                        self.stats[k] = v

    
    def get_upload_data(self):
        data = {'nodes':[], 'edges':[]}
        data['nodes'].append(self.design.get_data())
        for i in self.components:
            data['nodes'].append(i.get_data())
            data['edges'].append({'source':self.objid, 'target':i.objid, 'label':'has'})
        data['nodes'].append(self.get_data())
        return data

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.stats.items():
            fund[k] = v
        fund['type'] = self.type
        return fund



In [197]:
ship = Ship(probe, ship_configurations['components'])
ship

<ship: probe; 9827306495835; Probe>

In [198]:
ship.components

[<component: engine; 6842355547894; Engine>,
 <component: scanner; 8102588717084; Scanner>]

In [199]:
ship.stats

{'speed': 1, 'build_effort': 2, 'cost': 2}

In [200]:
ship.get_data()

{'name': 'Probe',
 'objid': '9827306495835',
 'label': 'ship',
 'speed': 1,
 'build_effort': 2,
 'cost': 2,
 'type': 'probe'}

In [201]:
pd.DataFrame(ship.get_upload_data()['nodes'])

name          objid      label     type  \
0    Probe  1713445880684     design    probe   
1   Engine  6842355547894  component   engine   
2  Scanner  8102588717084  component  scanner   
3    Probe  9827306495835       ship    probe   

                                         description         components  \
0  One way scanning machine to observe a location...  [engine, scanner]   
1  Engines for the ship. Increases the speed and ...                NaN   
2               Scans the environment for resources.                NaN   
3                                                NaN                NaN   

                          augments_ship_stats  fuel render grants_capability  \
0                                         NaN   NaN    NaN               NaN   
1  {'speed': 1, 'build_effort': 1, 'cost': 1}   0.0   cone               NaN   
2              {'cost': 1, 'build_effort': 1}   NaN   ball            [scan]   
3                                         NaN   NaN    NaN               NaN   

   speed  build_effort  cost  
0    NaN           NaN   NaN  
1    NaN           NaN   NaN  
2    NaN           NaN   NaN  
3    1.0           2.0   2.0

In [194]:
pd.DataFrame(ship.get_upload_data()['edges'])

source         target label
0  0079912752816  7283795160076   has
1  0079912752816  1345294256743   has

## The Construction process:
1. user will click on the `build_ship` button in the UX. 
2. ajax will take the building info and create a task.
3. a job is created for the build
4. upon completion, the ship is added to whichever `shipyard` exists in that faction